## Ordinary Least Square Model

In [69]:


import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
import numpy as np
import scipy.stats as ss
import pandas as pd
import statsmodels.api as sm

from pandas_datareader import DataReader
from datetime import datetime





### Defining Y variable as Tesla and adding  features 

In [58]:

TSLA = DataReader('TSLA',  'yahoo', datetime(2021,1,1), datetime(2021,8,30))
Y = np.diff(np.log(TSLA['Adj Close'].values))
T = Y.shape[0];



F = DataReader('SPY',  'yahoo', datetime(2021,1,1), datetime(2021,8,30))
F1 = np.diff(np.log(F['Adj Close'].values))
QQQ = DataReader('QQQ',  'yahoo', datetime(2021,1,1), datetime(2021,8,30))
F2 = np.diff(np.log(QQQ['Adj Close'].values))


### Performing OLS Regression Analysis

In [60]:

'Add Constant to X'
X = np.column_stack([np.ones(T),np.linspace(1,T,T), F1, F2])
N = X.shape[1]

'OLS REGRESSION STARTS'
'Linear Regression of Y: T x 1 on'
'Regressors X: T x N'
invXX = np.linalg.inv(X.transpose()@X)
'OLS estimates for coefficients: X x 1'
beta_hat = invXX@X.transpose()@Y
'Predictive value of Y using OLS'
y_hat = X@beta_hat
'Residuals from OLS'
residuals = Y - y_hat
'Variance of residuals'
sigma2 = (1/T)*residuals.transpose()@residuals
'standard deviation of Y or residuals'
sigma = np.sqrt(sigma2)







"'variance-covariance matrix of beta_hat'"

varcov_beta_hat = (sigma2)*invXX
std_beta_hat = np.sqrt(T*np.diag(varcov_beta_hat))

'Calculate R-square'
R_square = 1- (residuals.transpose()@residuals)/(T*np.var(Y))
adj_R_square = 1-(1-R_square)*(T-1)/(T-N)

'Test Each Coefficient: beta_i'
'Null Hypothesis: beta_i = 0'
t_stat = (beta_hat.transpose()-0)/std_beta_hat
p_val_t = 1-ss.norm.cdf(t_stat)

'Test of Joint Significance of Model'
F_stat = (beta_hat.transpose()@np.linalg.inv(varcov_beta_hat)@beta_hat/N)/\
         (residuals.transpose()@residuals/(T-N))

p_val_F = 1-ss.f.cdf(F_stat,N-1,T-N)


REPORT = np.column_stack([beta_hat, t_stat,p_val_t])

print('------------------------\n')
print(' REGRESSION STATISTICS  \n')
print('------------------------\n')
print(pd.DataFrame(REPORT,columns=['Beta','t statistics','p value']))
print('------------------------\n')
print('------------------------\n')
print('\n Joint significance of all coefficients\n')
print(pd.DataFrame(np.round([F_stat,p_val_F],6),columns=['Joint Signifinance'],index=['F stat','p value']))
print('------------------------\n')
print('\n R squared coefficients\n')
print(pd.DataFrame(np.round([R_square,adj_R_square,sigma,T],6),columns=['R Coefficents'],index=['R-Square','Adjusted R Square','Standard Error','Observations']))
print('------------------------\n')





------------------------

 REGRESSION STATISTICS  

------------------------

       Beta  t statistics   p value
0 -0.001708     -0.034336  0.513695
1 -0.000001     -0.002120  0.500846
2 -1.352687     -0.221893  0.587801
3  2.791040      0.677657  0.248995
------------------------

------------------------


 Joint significance of all coefficients

         Joint Signifinance
F stat         64438.749229
p value            0.000000
------------------------


 R squared coefficients

                   R Coefficents
R-Square                0.490375
Adjusted R Square       0.480879
Standard Error          0.024516
Observations          165.000000
------------------------



### Observations:
#### Coefficents and p-values for all the features is more than 5% which tells us that the market returns are not significantly influenced by dynamics of Tesla (Intercept is not significant)
#### Joint significance performed well as p-value is less than 5%
#### R squared is low, which shows the market aggregate can only determine 49% of the Data Dyamics

### Fama French Factor Model

In [65]:
import getFamaFrenchFactors as gff


#stock_data = yf.download(ticker, start, end, adjusted=True)
TSLA = DataReader('TSLA',  'yahoo', datetime(2018,1,1), datetime(2021,8,30))

ff3_monthly = gff.famaFrench3Factor(frequency='m')
ff3_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace=True)

In [66]:

stock_returns = TSLA['Adj Close'].resample('M').last().pct_change().dropna()
stock_returns.name = "Month_Rtn"
ff_data = ff3_monthly.merge(stock_returns,on='Date')

In [96]:
X = ff_data[['Mkt-RF', 'SMB', 'HML']]
y = ff_data['Month_Rtn'] - ff_data['RF']
'Add Constant to X'
X.insert(loc=0,value=np.ones(len(X)),column='I')
N = X.shape[1]

'Linear Regression of Y: T x 1 on'
'Regressors X: T x N'
invXX = np.linalg.inv(X.transpose()@X)
'OLS estimates for coefficients: X x 1'
beta_hat = invXX@X.transpose()@y


In [92]:
rf = ff_data['RF'].mean()
market_premium = ff3_monthly['Mkt-RF'].mean()
size_premium = ff3_monthly['SMB'].mean()
value_premium = ff3_monthly['HML'].mean()

expected_monthly_return = rf + b1 * market_premium + b2 * size_premium + b3 * value_premium 
expected_yearly_return = expected_monthly_return * 12
print("Expected yearly return: " + str(expected_yearly_return))

Expected yearly return: 0.1551753598202423


In [95]:
ff_model